In [ ]:
!apt-get install tesseract-ocr -y
!pip install pytesseract transformers torch torchvision --quiet

In [ ]:
import torch
import pytesseract
from pytesseract import Output
from transformers import LayoutLMv3Processor, LayoutLMv3ForQuestionAnswering
from PIL import Image
import json
import re

In [ ]:
model_path = "/content"

processor = LayoutLMv3Processor.from_pretrained(model_path)
model = LayoutLMv3ForQuestionAnswering.from_pretrained(model_path)

model.to(device)
model.eval()

In [ ]:
def get_ocr_words_boxes(image_path):

    image = Image.open(image_path).convert("RGB")
    width, height = image.size

    data = pytesseract.image_to_data(image, output_type=Output.DICT)

    words = []
    boxes = []

    n = len(data["text"])

    for i in range(n):
        if int(data["conf"][i]) > 0 and data["text"][i].strip() != "":
            word = data["text"][i]
            x = data["left"][i]
            y = data["top"][i]
            w = data["width"][i]
            h = data["height"][i]

            x0 = int(1000 * x / width)
            y0 = int(1000 * y / height)
            x1 = int(1000 * (x + w) / width)
            y1 = int(1000 * (y + h) / height)

            words.append(word)
            boxes.append([x0, y0, x1, y1])

    return words, boxes

In [ ]:
field_questions = {
    "hospital_name": "What is the hospital name?",
    "bill_number": "What is the bill number?",
    "bill_date": "What is the bill date?",
    "admit_date": "What is the admit date?",
    "discharge_date": "What is the discharge date?",
    "mrn": "What is the MRN number?",
    "total_amount": "What is the total amount?"
}

In [ ]:
import torch

def extract_field(image_path, field_name):

    question = field_questions[field_name]
    words, boxes = get_ocr_words_boxes(image_path)

    image = Image.open(image_path).convert("RGB")

    encoding = processor(
        images=image,
        text=question,
        text_pair=words,
        boxes=boxes,
        padding="max_length",
        truncation=True,
        return_tensors="pt"
    )

    sequence_ids = encoding.sequence_ids(0)
    encoding = {k: v.to(device) for k, v in encoding.items()}

    with torch.no_grad():
        outputs = model(**encoding)

    start_logits = outputs.start_logits.squeeze(0)
    end_logits = outputs.end_logits.squeeze(0)

    # Mask question tokens
    for i, seq_id in enumerate(sequence_ids):
        if seq_id != 1:
            start_logits[i] = -1e9
            end_logits[i] = -1e9

    start_pred = torch.argmax(start_logits).item()
    end_pred = torch.argmax(end_logits).item()

    tokens = processor.tokenizer.convert_ids_to_tokens(
        encoding["input_ids"].squeeze(0)
    )

    predicted_tokens = tokens[start_pred:end_pred+1]
    predicted_text = processor.tokenizer.convert_tokens_to_string(predicted_tokens)

    return predicted_text.strip()

In [ ]:
def extract_document_fields(image_path):

    extracted = {}

    for field in field_questions:
        try:
            value = extract_field(image_path, field)
            extracted[field] = value
        except:
            extracted[field] = None

    return extracted


In [ ]:
image_path = "/content/2048528-06.jpg"

result = extract_document_fields(image_path)

import json
print(json.dumps(result, indent=2))